In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import os
import zipfile
import datetime
import shutil

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [ ]:
local_zip = '/content/drive/MyDrive/DL_dataset2/data.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

In [ ]:
print(len(os.listdir('/content/train/Signed')))
print(len(os.listdir('/content/train/Unsigned')))
print(len(os.listdir('/content/val/Signed')))
print(len(os.listdir('/content/val/Unsigned')))
print(len(os.listdir('/content/train/sign')))

23489
23489
1000
1000
25000


In [ ]:
dir_1 = '/content/train/Signed'
dir_2 = '/content/train/sign'

dir1_lst = os.listdir(dir_1)
dir1_lst.sort()

for img in dir1_lst[:25000]:
  shutil.move(os.path.join(dir_1, img), os.path.join(dir_2, img))

dir_3 = '/content/val/Sign'

for img in dir1_lst[25000:]:
  shutil.move(os.path.join(dir_1, img), os.path.join(dir_3, img))

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1/255)
validation_data_gen = ImageDataGenerator(rescale=1/255)

train_gen = train_data_gen.flow_from_directory(
    '/content/train',
    target_size = (256,256),
    batch_size = 32,
    class_mode = 'binary'
)

val_gen = train_data_gen.flow_from_directory(
    '/content/val',
    target_size = (256,256),
    batch_size = 16,
    class_mode = 'binary'
)

Found 46978 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
train_data_gen = ImageDataGenerator(rescale=1/255)
validation_data_gen = ImageDataGenerator(rescale=1/255)

train_gen = train_data_gen.flow_from_directory(
    '/content/train',
    target_size = (256,256),
    batch_size = 32,
    class_mode = 'binary'
)

val_gen = train_data_gen.flow_from_directory(
    '/content/val',
    target_size = (256,256),
    batch_size = 16,
    class_mode = 'binary'
)

In [ ]:
model = Sequential()
model.add(layers.Conv2D(input_shape=(256, 256, 3), filters=64,kernel_size=(3,3), padding="same",activation='relu'))
model.add(layers.Conv2D(filters=32,kernel_size=(3,3), padding="same",activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 126, 126, 128)     36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 128)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 256)       2

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics = ['accuracy'])

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=5)

In [ ]:
model_fit = model.fit(train_gen,
                     epochs = 20,
                     verbose = 1,
                      steps_per_epoch=100,
                     validation_data = val_gen,
                      callbacks = early_stopping)


Epoch 1/20
100/100 [==============================] - 41s 256ms/step - loss: 0.5093 - accuracy: 0.7688 - val_loss: 0.6020 - val_accuracy: 0.7565
Epoch 2/20
100/100 [==============================] - 25s 249ms/step - loss: 0.4377 - accuracy: 0.8238 - val_loss: 0.5498 - val_accuracy: 0.7605
Epoch 3/20
100/100 [==============================] - 25s 252ms/step - loss: 0.3887 - accuracy: 0.8450 - val_loss: 0.5881 - val_accuracy: 0.7410
Epoch 4/20
100/100 [==============================] - 26s 254ms/step - loss: 0.3902 - accuracy: 0.8350 - val_loss: 0.4320 - val_accuracy: 0.8230
Epoch 5/20
100/100 [==============================] - 26s 258ms/step - loss: 0.3329 - accuracy: 0.8722 - val_loss: 0.4722 - val_accuracy: 0.7990
Epoch 6/20
100/100 [==============================] - 26s 259ms/step - loss: 0.3101 - accuracy: 0.8844 - val_loss: 0.3175 - val_accuracy: 0.8850
Epoch 7/20
100/100 [==============================] - 26s 259ms/step - loss: 0.3131 - accuracy: 0.8806 - val_loss: 0.4437 - val_ac

In [ ]:
model.save('model1.h5')

In [ ]:
local_zip = '/content/drive/MyDrive/DL_dataset2/inference.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

In [ ]:
inf_model = models.load_model('/content/model1.h5')

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1/255)
validation_data_gen = ImageDataGenerator(rescale=1/255)

infer_gen = train_data_gen.flow_from_directory(
    '/content/infer',
    target_size = (256,256),
    batch_size = 32,
    class_mode = 'binary'
)


Found 6000 images belonging to 2 classes.


In [ ]:
score = inf_model.evaluate(infer_gen, verbose=1)

188/188 [==============================] - 15s 78ms/step - loss: 0.1636 - accuracy: 0.9482


In [ ]:
import cv2
import numpy as np
from os import listdir

dir_sign = '/content/train/Signed'
dir_sign_list = listdir(dir_sign)
dir_sign_list.sort()

wpx = 0
for img in dir_sign_list:
  img = cv2.imread(os.path.join(dir_sign,img))
  wcount = (np.count_nonzero(img)/img.size)*100
  wpx = wpx + count

avg_wpx = wpx/len(dir_sign_list)
print(avg_wpx)

59.20936771798022


In [ ]:
import cv2
import numpy as np
from os import listdir

dir_unsign = '/content/train/Unsigned'
dir_unsign_list = listdir(dir_unsign)
dir_unsign_list.sort()

bpx = 0
for img in dir_unsign_list:
  img = cv2.imread(os.path.join(dir_unsign,img))
  bcount = (np.count_nonzero(img)/img.size)*100
  bpx = bpx + count

avg_bpx = bpx/len(dir_unsign_list)
print(avg_bpx)

49.53740820015236


In [ ]:
dir_sign = '/content/train/Signed'
dir_sign_list = os.listdir(dir_sign)
dir_sign_list.sort()
crct = 0
incrt = 0
for image_nm in dir_sign_list:
  img = cv2.imread(os.path.join(dir_sign,image_nm))
  wcount = (np.count_nonzero(img)/img.size)*100
  if wcount >= 59:  crct = crct +1
  else: incrt = incrt +1

print("Correctly classified:",crct,"\tIncorrectly classified:",incrt)

Correctly classified: 13923 	Incorrectly classified: 9566


In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.5 MB 824 kB/s 


In [ ]:
from google.colab.patches import cv2_imshow
import math
import numpy as np

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480
def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2_imshow(img)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in uploaded.keys()}
# Preview the images.
for name, image in images.items():
  print(name)   
  resize_and_show(image)

In [ ]:
import mediapipe as mp
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

with mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:
  for name, image in images.items():
    # Convert the BGR image to RGB and process it with MediaPipe Pose.
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # Print nose landmark.
    image_hight, image_width, _ = image.shape
    if not results.pose_landmarks:
      continue
    print(
      f'Wrist coordinates: ('
      f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * image_width}, '
      f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * image_hight})'
    )

    # Draw pose landmarks.
    print(f'Pose landmarks of {name}:')
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    resize_and_show(annotated_image)

In [ ]:
# Run MediaPipe Pose and draw pose landmarks.
with mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:
  for name, image in images.items():
    # Convert the BGR image to RGB and process it with MediaPipe Pose.
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # Print nose landmark.
    image_hight, image_width, _ = image.shape
    if not results.pose_landmarks:
      continue
    print(
      f'Wrist coordinates: ('
      f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width}, '
      f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image_hight})'
    )

    # Draw pose landmarks.
    print(f'Pose landmarks of {name}:')
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    resize_and_show(annotated_image)

In [ ]:
# Run MediaPipe Pose and plot 3d pose world landmarks.
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:
  for name, image in images.items():
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    #3D coordinates of nose in meters with the origin at the center between hips.
    print('Nose world landmark:')
    print(results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE])
    
    # Plot pose world landmarks.
    mp_drawing.plot_landmarks(
        results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)